In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm.notebook')

import os
import ast
import json

import torch

import numpy as np
import pandas as pd
from tqdm import tqdm

from utilities import Generator
from utilities import promptGenerator

from databench_eval import Evaluator
from databench_eval.utils import load_qa, load_table

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
device     = torch.device("cuda:3")  # Select GPU 3
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct" # "google/gemma-3-4b-it"
gen        = Generator.Generator(model_name, device)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.62it/s]
Device set to use cuda:3


In [3]:
qa_dev = load_qa(lang="ES", name="iberlef", split="dev")

In [4]:
def save_responses(responses, save_path: str) -> None:
    with open(save_path, "w") as f:
        for response in responses:
            f.write(str(response).replace("\n", " ") + "\n")

In [5]:
def example_postprocess(response: str, row: dict):
    try:
        df = load_table(row["dataset"], lang="ES")
        
        global ans
        lead = """
def answer(df):
    """
        exec_string = (
            response
            + "\nans = answer(df)"
        )
        local_vars = {"df": df, "pd": pd, "np": np}
        exec(exec_string, local_vars)

        ans = local_vars["ans"]
        if isinstance(ans, pd.Series):
            ans = ans.tolist()
        elif isinstance(ans, pd.DataFrame):
            ans = ans.iloc[:, 0].tolist()
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [6]:
responses = []

for instance in tqdm(qa_dev):
    prompt_handler = promptGenerator.promptGenerator(instance, True)

    # GET COLUMNS
    prompt_columns = prompt_handler.build_full_prompt(prompt_handler.getColumns(), "columns")
    columns        = gen.getLLMOutput(prompt_columns, 150, True, 0.5, 0.7)
    prompt_handler.setReducedColumns(columns)
    
    # GET INSTRUCTIONS
    prompt_instructions = prompt_handler.build_full_prompt(prompt_handler.getInstructions(columns), "instructions")
    cot_instructions    = gen.getLLMOutput(prompt_instructions, 400, True, 0.6, 0.8)

    # GET CODE
    prompt_code = prompt_handler.build_full_prompt(prompt_handler.getCode(cot_instructions, columns), "code")
    code        = gen.getLLMOutput(prompt_code, 500, True, 0.4, 0.8)

    # RUN CODE
    output_code = example_postprocess(code, instance)

    # DEBUG CODE
    if str(output_code).startswith("__CODE_ERROR__"):
        attempts = 1
        while str(output_code).startswith("__CODE_ERROR__") and attempts <= 2:
            # GET "FIXED" CODE
            prompt_fix_bug = prompt_handler.build_full_prompt(prompt_handler.getCorrectCode(cot_instructions, columns, code, output_code), "code_correction")
            fixed_code     = gen.getLLMOutput(prompt_fix_bug, 600, True, 0.5, 0.8)
            output_code    = example_postprocess(fixed_code, instance)
            attempts += 1

    responses.append([str(output_code)])

  0%|                                                                                                                                                            | 0/100 [00:00<?, ?it/s]

¿Hay más mujeres que hombres en esta encuesta?
['Sexo', '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?']
To answer the question "¿Hay más mujeres que hombres en esta encuesta?", the high-level steps are:

1. **Filter the DataFrame**: Extract the relevant data from the DataFrame, specifically the 'Sexo' column.
2. **Count the occurrences**: Count the number of unique values in the 'Sexo' column, which represent the number of women and men.
3. **Compare counts**: Determine which count is higher, women or men.
4. **Return a boolean result**: Return a boolean value indicating whether there are more women than men in the survey.
def answer(df: pd.DataFrame):
    """Explain each step with comments above."""
    # Select the column with the responses to the 'Sexo' question
    sexo_column = 'Sexo'
    
    # Filter the rows where the value in the 'Sexo' column is 'Mujer' or 'Hombre'
    sexo_values = df[sexo_column]
    
    # Count the number of unique values in the

  1%|█▍                                                                                                                                                  | 1/100 [00:17<29:01, 17.59s/it]

¿Es 18-24 el grupo de edad más frecuente entre los hombres?
['Sexo', 'Edad']
To answer the question, follow these high-level steps:

1. **Filter the DataFrame**: Extract a subset of rows where the 'Sexo' column matches 'Hombre'.
2. **Group the filtered DataFrame**: Group the resulting DataFrame by the 'Edad' column.
3. **Calculate the frequency of each age group**: Determine the count of rows for each age group.
4. **Identify the most frequent age group**: Find the age group with the highest frequency.
5. **Check if the most frequent age group is 18-24**: Compare the most frequent age group to the specified age range (18-24).
6. **Return the result as a boolean**: Indicate whether the most frequent age group matches the specified age range.
def answer(df: pd.DataFrame):
    """Check if 18-24 is the most frequent age group among men."""
    # Filter the rows where the value in the 'Sexo' column is 'Hombre'
    hombres = df[df['Sexo'] == 'Hombre']
    
    # Group the filtered DataFrame 


  2%|██▉                                                                                                                                                 | 2/100 [00:35<28:40, 17.56s/it]

...

¿Cuál es el número más común de hijos que le hubiera gustado o gustaría tener a los participantes?
['¿Cuántos hijos te habría gustado o te gustaría tener?', '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?']
To address the question, follow these high-level steps:

1. **Data Preparation**: Ensure the relevant columns ['¿Cuántos hijos te habría gustado o te gustaría tener?' and '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?'] are correctly formatted and free of missing or inconsistent data.

2. **Data Extraction**: Extract the data from the relevant columns into a new data structure, likely a pandas Series or a new DataFrame, to facilitate analysis.

3. **Data Cleaning**: Remove any non-numeric data from the extracted data, as it may not be relevant to the question or may cause analysis errors.

4. **Data Aggregation**: Group the cleaned data by the extracted values and calculate the frequency of each value.

5. **Data Analysis**: Identif


 10%|██████████████▋                                                                                                                                    | 10/100 [04:30<55:13, 36.82s/it]

DataBenchSPA accuracy is 0.06
¿Cuál es el número más alto de hijos que le hubiera gustado o gustaría tener a algún participante?
['¿Cuántos hijos te habría gustado o te gustaría tener?', '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?']
To answer the question, follow these high-level steps:

1. **Identify relevant columns**: Select the columns '¿Cuántos hijos te habría gustado o te gustaría tener?' and '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?' from the DataFrame.
2. **Extract numerical values**: Extract the numerical values from the selected columns, assuming they contain string representations of numbers.
3. **Convert to numeric data type**: Convert the extracted values to a numeric data type (e.g., int or float) to enable mathematical operations.
4. **Find the maximum value**: Identify the maximum value among the converted numerical values.
5. **Return the result**: Return the maximum value as the answer to the question.


 10%|██████████████▋                                                                                                                                    | 10/100 [04:48<43:15, 28.84s/it]


KeyboardInterrupt: 

In [ ]:
evaluator = Evaluator(qa=qa_dev)
print(f"DataBenchSPA accuracy is {evaluator.eval(responses)}")
save_responses(responses, "predictions.txt")